In [ ]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt



In [ ]:
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

trade_active = mt5.initialize()

print(trade_active)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4200
True


In [ ]:
if not trade_active:
    print('Initialization failed, check internet connection. You must have Meta Trader 5 installed.')
    mt5.shutdown()

else:
    print(mt5.account_info()._asdict())
    print("\n")
    print(mt5.terminal_info()._asdict())
    print("\n")
    print(mt5.symbols_total())



{'login': 213644473, 'trade_mode': 0, 'leverage': 1000, 'limit_orders': 500, 'margin_so_mode': 0, 'trade_allowed': True, 'trade_expert': True, 'margin_mode': 2, 'currency_digits': 2, 'fifo_close': False, 'balance': 200.0, 'credit': 0.0, 'profit': 0.0, 'equity': 200.0, 'margin': 0.0, 'margin_free': 200.0, 'margin_level': 0.0, 'margin_so_call': 25.0, 'margin_so_so': 15.0, 'margin_initial': 0.0, 'margin_maintenance': 0.0, 'assets': 0.0, 'liabilities': 0.0, 'commission_blocked': 0.0, 'name': 'micheal bright omage ', 'server': 'OctaFX-Demo', 'currency': 'USD', 'company': 'Octa Markets Incorporated'}


{'community_account': False, 'community_connection': False, 'connected': True, 'dlls_allowed': False, 'trade_allowed': True, 'tradeapi_disabled': False, 'email_enabled': False, 'ftp_enabled': False, 'notifications_enabled': False, 'mqid': False, 'build': 5200, 'maxbars': 100000, 'codepage': 0, 'ping_last': 150601, 'community_balance': 0.0, 'retransmission': 0.6890690676895515, 'company': 'Meta

In [ ]:
account = mt5.account_info()
terminal = mt5.terminal_info()

print(account.equity)

if(terminal.connected == True and terminal.trade_allowed == True):
    print("AI is successfully functional")
else:
    print("Please make sure metatrade 5 has internet and algo Trade is Turn On")

200.0
AI is successfully functional


In [ ]:
symbols = mt5.symbols_get()
print(len(symbols))
t_symbol = ["GBPUSD","EURUSD","USDCAD","USDJPY","AUDUSD"]
for symbol in symbols:
    for t_s in t_symbol:
        if(symbol.name == t_s):
            print("yes ",t_s)
            break



280
yes  EURUSD
yes  GBPUSD
yes  USDJPY
yes  AUDUSD
yes  USDCAD


In [ ]:
n = 0

In [ ]:
market = mt5.copy_rates_from_pos(t_symbol[n], mt5.TIMEFRAME_M30, 0, 99999)

In [ ]:
market

array([(1503012600, 1.28688, 1.28692, 1.28629, 1.28644, 1504,  8, 0),
       (1503014400, 1.28668, 1.28693, 1.28656, 1.2868 ,  183, 13, 0),
       (1503016200, 1.28677, 1.28694, 1.28643, 1.28664, 2022, 24, 0), ...,
       (1756506600, 1.35073, 1.35148, 1.35073, 1.35147, 1963, 13, 0),
       (1756508400, 1.35148, 1.35164, 1.35101, 1.35124, 1228, 13, 0),
       (1756510200, 1.35125, 1.35126, 1.34995, 1.35018, 2122, 13, 0)],
      dtype=[('time', '<i8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('tick_volume', '<u8'), ('spread', '<i4'), ('real_volume', '<u8')])

In [ ]:
len(market)

99999

In [ ]:
data = []
for i in range(len(market)):
    data.append([market[i][1],market[i][2],market[i][3],market[i][4]])

In [ ]:
df = pd.DataFrame(data, columns=["open", "high","low", "close"])
display(df)

,open,high,low,close
0,1.28688,1.28692,1.28629,1.28644
1,1.28668,1.28693,1.28656,1.28680
2,1.28677,1.28694,1.28643,1.28664
3,1.28667,1.28741,1.28647,1.28671
4,1.28672,1.28736,1.28669,1.28711
...,...,...,...,...
99994,1.35005,1.35105,1.35003,1.35095
99995,1.35093,1.35114,1.35050,1.35073
99996,1.35073,1.35148,1.35073,1.35147
99997,1.35148,1.35164,1.35101,1.35124


In [ ]:
df.to_csv("Generated"+t_symbol[n]+" dbot.csv", index=False)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf


In [2]:
train_df = pd.read_csv("GeneratedGBPUSD dbot.csv")
display(train_df)

,open,high,low,close
0,1.28688,1.28692,1.28629,1.28644
1,1.28668,1.28693,1.28656,1.28680
2,1.28677,1.28694,1.28643,1.28664
3,1.28667,1.28741,1.28647,1.28671
4,1.28672,1.28736,1.28669,1.28711
...,...,...,...,...
99994,1.35005,1.35105,1.35003,1.35095
99995,1.35093,1.35114,1.35050,1.35073
99996,1.35073,1.35148,1.35073,1.35147
99997,1.35148,1.35164,1.35101,1.35124


In [3]:
# -------------------------
# User-adjustable settings
# -------------------------
SEQ_LEN = 48              # length of input sequence (timesteps). Typical choices: 30, 60, 90
VAL_SIZE = 0.1           # fraction used as validation (small, for fine-tuning)
MODEL_SAVE_PATH = 'lstm_multivar_best.h5'

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

In [5]:
def create_sequences(data_scaled, seq_len):
    """
    data_scaled: np.array shaped (n_rows, n_features)
    returns X (n_samples, seq_len, n_features), y (n_samples, n_features)
    where y is the row immediately following the window.
    """
    X, y = [], []
    n_rows = data_scaled.shape[0]
    for i in range(n_rows - seq_len):
        X.append(data_scaled[i:i+seq_len])
        y.append(data_scaled[i+seq_len])
    X = np.array(X)
    y = np.array(y)
    return X, y

# 1) Convert to numpy and ensure numeric
n_rows, n_features = train_df.values.shape
"""
# 2) Fit scaler on all feature columns (important to use same scaler for inverse transform)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_all = scaler.fit_transform(train_df.values)  # (n_rows, n_features)
"""
# 3) Build sequences
X, y = create_sequences(train_df.values, SEQ_LEN)  # X: (n_samples, SEQ_LEN, n_features), y: (n_samples, n_features)

n_samples = X.shape[0]


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=None, shuffle=False)

In [8]:
X_test, x_val, y_test,  y_val = train_test_split(X_test, y_test, test_size=0.45, random_state=None, shuffle=False)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)

(74963, 48, 4)
(13743, 48, 4)
(74963, 4)
(13743, 4)
(11245, 48, 4)
(11245, 4)


In [10]:
import concurrent.futures
import os, json, math
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Bidirectional, BatchNormalization, LayerNormalization

In [11]:
# ------------------------
# Settings
# ------------------------
RANDOM_STATE = 42
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

MODEL_SAVE_PATH = "lstm_tuned_best.h5"
HP_SAVE_PATH = "lstm_tuned_best_hparams.json"
MAX_EPOCHS = 100
N_RANDOM_TRIALS = 15
NO_IMPROVE_ROUNDS = 10

ES_PATIENCE = 12
RLR_PATIENCE = 6

In [12]:

# Assumed data already prepared
try:
    X_train, y_train, x_val, y_val
except NameError:
    raise RuntimeError("Please prepare X_train, y_train, X_val, y_val before running this tuner script.")

seq_len = X_train.shape[1]
n_features = X_train.shape[2]
output_dim = y_train.shape[1]
assert output_dim == n_features, "For multi-output forecasting, y must match X features."

In [13]:

# ------------------------
# Model Builder
# ------------------------
def build_model_from_cfg(cfg):
    i = Input(shape=(seq_len, n_features))
    x = i
    for li in range(cfg["num_lstm_layers"]):
        units = cfg["units_per_layer"][li]
        return_sequences = (li < cfg["num_lstm_layers"] - 1)
        if cfg["bidirectional"]:
            lstm_layer = Bidirectional(
                LSTM(units, return_sequences=return_sequences,
                     recurrent_dropout=cfg["recurrent_dropout"])
            )
        else:
            lstm_layer = LSTM(units, return_sequences=return_sequences,
                              recurrent_dropout=cfg["recurrent_dropout"])
        x = lstm_layer(x)
        if cfg["normalization"] == "batch":
            x = BatchNormalization()(x)
        elif cfg["normalization"] == "layer":
            x = LayerNormalization()(x)
        if cfg["dropout"] > 0:
            x = Dropout(cfg["dropout"])(x)

    if cfg["dense_head"] > 0:
        x = Dense(cfg["dense_head"], activation="relu")(x)
        if cfg["dropout_head"] > 0:
            x = Dropout(cfg["dropout_head"])(x)

    out = Dense(output_dim)(x)
    model = Model(i, out)
    opt = tf.keras.optimizers.Adam(learning_rate=cfg["lr"])
    model.compile(optimizer=opt, loss="mse", metrics=["mae"])
    return model


# ------------------------
# Config Sampler
# ------------------------
def sample_config(rng):
    num_lstm_layers = rng.randint(1, 5)
    candidate_units = [64, 96, 128, 160, 192, 256, 320, 384, 512]
    first_units = rng.choice(candidate_units)
    units_per_layer = [first_units]
    for _ in range(1, num_lstm_layers):
        allowed = [u for u in candidate_units if u <= units_per_layer[-1]]
        units_per_layer.append(rng.choice(allowed))

    return {
        "num_lstm_layers": num_lstm_layers,
        "units_per_layer": units_per_layer,
        "bidirectional": bool(rng.randint(0, 2)),
        "dropout": float(rng.uniform(0.1, 0.5)),
        "recurrent_dropout": float(rng.uniform(0.0, 0.3)),
        "normalization": rng.choice(["none", "batch", "layer"]),
        "dense_head": int(rng.choice([0, 64, 96, 128, 160, 192, 256])),
        "dropout_head": float(rng.uniform(0.0, 0.5)),
        "lr": float(rng.choice([1e-4, 3e-4, 5e-4, 1e-3])),
        "batch_size": int(rng.choice([8, 16, 32, 48, 64])),
        "epochs": int(rng.choice([60, 80, 100, 120]))
    }


# ------------------------
# Worker: train a config
# ------------------------
def train_one_config(cfg, trial_id):
    model = build_model_from_cfg(cfg)
    es = EarlyStopping(monitor='val_loss', patience=ES_PATIENCE, restore_best_weights=True, verbose=2)
    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=RLR_PATIENCE, min_lr=1e-6, verbose=2)
    mc_path = f"trial_{trial_id}_best.h5"

    mc = ModelCheckpoint(mc_path, monitor='val_loss', save_best_only=True, verbose=2)

    hist = model.fit(
        X_train, y_train,
        validation_data=(x_val, y_val),
        epochs=min(cfg["epochs"], MAX_EPOCHS),
        batch_size=cfg["batch_size"],
        callbacks=[es, rlr, mc],
        verbose=2
    )

    best_vloss = float(np.min(hist.history["val_loss"]))
    return best_vloss, cfg, mc_path




In [ ]:
rng = np.random.RandomState(RANDOM_STATE)
configs = [sample_config(rng) for _ in range(N_RANDOM_TRIALS)]

best_vloss = math.inf
best_cfg = None
best_model_path = None

with concurrent.futures.ProcessPoolExecutor(max_workers=os.cpu_count()//2) as executor:
    futures = {executor.submit(train_one_config, cfg, i): i for i, cfg in enumerate(configs, 1)}
    for future in concurrent.futures.as_completed(futures):
        trial_id = futures[future]
        try:
            vloss, cfg, mc_path = future.result()
            print(f"[Trial {trial_id}] val_loss={vloss:.6f} cfg={cfg}")
            if vloss < best_vloss:
                best_vloss = vloss
                best_cfg = cfg
                best_model_path = mc_path
        except Exception as e:
            print(f"[Trial {trial_id}] failed with {e}")

if best_model_path and os.path.exists(best_model_path):
    os.replace(best_model_path, MODEL_SAVE_PATH)

with open(HP_SAVE_PATH, "w") as f:
    json.dump(best_cfg, f, indent=2)

print(f"\n✅ Best val_loss: {best_vloss:.6f}")
print(f"✅ Best model saved to: {MODEL_SAVE_PATH}")
print(f"✅ Best hyperparameters saved to: {HP_SAVE_PATH}")


[Trial 1] failed with Exception encountered when calling LSTM.call().

output_size must be an integer.

Arguments received by LSTM.call():
  • args=('<KerasTensor shape=(None, 48, 4), dtype=float32, sparse=False, ragged=False, name=keras_tensor>',)
  • kwargs={'training': 'False', 'mask': 'None'}
Epoch 1/60
